In [1]:
import numpy as np
from numba import njit

Предположим, что мы действительно имеем 5 каналов (B02, B03, B04, B08, B11) и для каждого пикселя есть информация по каждому из данных значений.
Далее пусть у нас есть несколько многоугольников, обозначающих поля (не умаляя общности, X и Y).
Поскольку могут существовать некоторые локальные закономерности, нам стоит разбить их на меньшие поля (A, B и C для X; D, E для Y).

Для случайно сгенерированных значений рассмотрим "адекватность" различных изменений.

In [51]:
class SubPolygon:
    def __init__(self, size, last_values = None, last_mean = 0, last_std = 4, 
                         current_values = None, current_mean = 5, current_std = 4):
        
        if not(last_values or current_values):
            self.last_values = np.random.normal(0, 4, (size, 5))
            self.current_values = np.random.normal(5, 4, (size, 5))
        else:
            self.last_values = last_values
            self.current_values = current_values
        
        self.change = self.current_values - self.last_values
        self.size = size
        
        self.mean_vec = np.mean(self.change, axis = 0)
        
    def get_changes(self):
        return (val for val in self.change)
    
    def mean(self):
        return self.mean_vec

In [54]:
class Polygon:
    def __init__(self, sub_list):
        self.list = sub_list
        self.size = sum([elem.size for elem in sub_list])
        self.mean_vec = np.sum([elem.size * elem.mean() for elem in sub_list]) / self.size
        
    def mean(self):
        return self.mean_vec

In [52]:
np.random.seed(42)

sub_polygons = {'A', 'B', 'C', 'D', 'E'}
polygons = {'X', 'Y'}
match_dict = {'X': {'A', 'B', 'C'}, 'Y': {'D', 'E'}}

lengths = {i:j for i,j in zip(sub_polygons, np.random.randint(1000, 10000, len(sub_polygons)))}
for polygon in polygons:
    lengths[polygon] = sum([lengths[child] for child in match_dict[polygon]])

In [53]:
lengths

{'E': 8270, 'B': 1860, 'C': 6390, 'D': 6191, 'A': 6734, 'Y': 14461, 'X': 14984}